In [16]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import math
from ortools.linear_solver import pywraplp
import folium
import time
import numpy as np
from sklearn.cluster import KMeans

In [17]:
pd.set_option('display.float_format', '{:.10f}'.format)

shp_file_path_15 = "Data\남구 65세 이상 기초수급자 거주지(통신사비율보정) - 복사본\남구 65세이상 기초수급자(통산사비율보정).shp"
df_Namgu = gpd.read_file(shp_file_path_15, encoding='cp949')

df_Namgu.head()
# len(df_Namgu)

,aver_price,demand,geometry
0,603211.6946988017,15,POINT (129.31711 35.52537)
1,771516.1970481118,12,POINT (129.32104 35.52112)
2,808496.6244030956,10,POINT (129.31717 35.52442)
3,914786.2726617508,13,POINT (129.3178 35.52161)
4,1009717.9426404362,9,POINT (129.31596 35.52304)


후보 시설 위치 집합 만들기 - k-means 알고리즘 사용

In [18]:
# GeoDataFrame 생성
gdf = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# 'geometry'에서 경도(longitude)와 위도(latitude) 추출
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

# X는 경도와 위도 값을 담고 있는 배열
X = gdf[['longitude', 'latitude']].values

# k-means 클러스터링 적용
k =100  # 원하는 클러스터 수
kmeans = KMeans(n_clusters=k, random_state=0)
gdf['cluster'] = kmeans.fit_predict(X)

# 각 클러스터의 중심 좌표 확인
centroids = kmeans.cluster_centers_
print("클러스터 중심 좌표: ", centroids)


클러스터 중심 좌표:  [[129.32564949  35.5424609 ]
 [129.26050828  35.54430577]
 [129.31709386  35.53613651]
 [129.33358849  35.52279066]
 [129.29648018  35.53611913]
 [129.33482989  35.53990729]
 [129.31222108  35.52732906]
 [129.32653886  35.53233939]
 [129.30766945  35.54564117]
 [129.31687577  35.51535787]
 [129.32232703  35.53479085]
 [129.34636385  35.53939205]
 [129.27259067  35.5490341 ]
 [129.30669142  35.53564082]
 [129.32185218  35.52945275]
 [129.33007984  35.53780188]
 [129.26127023  35.54812412]
 [129.30998151  35.54137796]
 [129.31258305  35.531268  ]
 [129.33023627  35.52506667]
 [129.3386214   35.53451706]
 [129.33269895  35.52929913]
 [129.3169804   35.52347163]
 [129.31346352  35.53752664]
 [129.3198451   35.54523307]
 [129.33967236  35.51957171]
 [129.33146207  35.54305377]
 [129.28977593  35.53382811]
 [129.32385738  35.53645288]
 [129.25731726  35.53791745]
 [129.28246691  35.5466979 ]
 [129.31436433  35.54524058]
 [129.33290899  35.53367864]
 [129.26394685  35.54971498]
 

c:\Users\jymin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [19]:
# import geopandas as gpd
# from shapely.geometry import Point

# # 클러스터 중심 좌표를 GeoDataFrame으로 변환
# gdf_centroids = gpd.GeoDataFrame(
#     {'cluster_id': range(len(centroids))},  # 클러스터 ID
#     geometry=[Point(x, y) for x, y in centroids],  # 중심 좌표를 포인트로 변환
#     crs="EPSG:4326"  # 좌표계 설정 (WGS84)
# )

# # 파일 저장 경로 설정
# output_path = r"C:\Users\jymin\OneDrive - UNIST (1)\OR LAB\(2024) Orienteering Problem\Code\Data\candidate_sites.shp"

# # GeoDataFrame을 shapefile로 저장
# gdf_centroids.to_file(output_path, driver="ESRI Shapefile")

In [20]:
# import geopandas as gpd
# from shapely.geometry import Point

# # Depot 좌표 추가
# depot_coords = [
#     (129.3299, 35.5299),  # df_Dosan의 Depot 좌표 -- 야음동
#     (129.2508, 35.5353),  # df_Moonsoo의 Depot 좌표 -- 무거동
#     (129.3187, 35.5140),  # df_Seonam의 Depot 좌표 -- 선암동
#     (129.3361, 35.5457)   # df_Ulsan의 Depot 좌표 -- 삼산동
# ]

# # NumPy 배열로 변환 후 centroids에 결합
# depot_coords = np.array(depot_coords)
# centroids = np.vstack([centroids, depot_coords])  # 수직으로 결합

# # GeoDataFrame 생성 (cluster_id는 1부터 시작)
# gdf_centroids = gpd.GeoDataFrame(
#     {'cluster_id': range(1, len(centroids) + 1)},  # 클러스터 ID
#     geometry=[Point(x, y) for x, y in centroids],  # 중심 좌표를 Point로 변환
#     crs="EPSG:4326"  # 좌표계 설정 (WGS84)
# )

# # 파일 저장 경로 설정
# output_path = r"C:\Users\jymin\OneDrive - UNIST (1)\OR LAB\(2024) Orienteering Problem\Code\Data\candidate_sites_v2.shp"

# # GeoDataFrame을 shapefile로 저장
# gdf_centroids.to_file(output_path, driver="ESRI Shapefile")
# print(f"총 {len(centroids)}개의 지점 데이터를 {output_path}에 저장했습니다.")


MCLP 모델 설정

In [21]:
# from scipy.spatial.distance import cdist
# from pulp import LpMaximize, LpProblem, LpVariable, lpSum
# import folium

# # 수요 데이터 (geometry 컬럼 사용)
# df_demand = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# # 후보 시설 위치 데이터 (클러스터링 결과로 나온 중심 좌표)
# facility_locations = np.array(centroids)

# # 시설 커버 범위 (~~정도의 거리 내에 있는 지점들을 커버한다고 가정)
# cover_range = 0.001 # 0.45~0.55 km

# # GeoDataFrame에서 좌표 추출 (수요 지점의 위도, 경도)
# demand_coordinates = np.array([[point.x, point.y] for point in gdf.geometry])

# # 유클리드 거리 계산 (수요 지점과 후보 시설 위치 간 거리)
# distances = cdist(demand_coordinates, facility_locations, metric='euclidean')

# # MCLP 모델 설정
# model = LpProblem(name="MCLP", sense=LpMaximize)

# # 변수 설정
# # x_j: 후보 시설 위치 j에 시설이 설치되면 1, 아니면 0
# x = [LpVariable(f"x_{j}", cat="Binary") for j in range(len(facility_locations))]

# # y_i: 수요 지점 i가 하나 이상의 시설에 의해 커버되면 1, 아니면 0
# y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(gdf))]

# # 목적 함수: 커버된 수요 지점의 총 수요값을 최대화
# model += lpSum(gdf['demand'][i] * y[i] for i in range(len(gdf)))

# # 제약조건 1: 수요 지점이 커버되려면 그 지점에서 커버 가능한 범위 내의 시설이 하나 이상 설치되어야 함
# for i in range(len(gdf)):
#     model += lpSum(x[j] for j in range(len(facility_locations)) if distances[i, j] <= cover_range) >= y[i]

# # 제약조건 2: 설치할 시설의 총 개수는 6개로 제한
# model += lpSum(x[j] for j in range(len(facility_locations))) == 6

# # 모델 최적화
# model.solve()

# # 결과 출력
# selected_facilities = [j for j in range(len(facility_locations)) if x[j].varValue == 1]
# covered_demand_points = [i for i in range(len(gdf)) if y[i].varValue == 1]

# print("선택된 시설 위치 인덱스: ", selected_facilities)
# print("선택된 시설 위치 좌표: ", facility_locations[selected_facilities])

# # 복지관 depot 위치 데이터
# depot_locations = [
#     (129.3299, 35.5299),  # df_Dosan의 Depot 좌표   --야음동
#     (129.2508, 35.5353),  # df_Moonsoo의 Depot 좌표 -- 무거동
#     (129.3187, 35.5140),  # df_Seonam의 Depot 좌표  -- 선암동
#     (129.3361, 35.5457)   # df_Ulsan의 Depot 좌표   -- 삼산동
# ]

# # 지도 생성
# # 지도 초기 중심
# m = folium.Map(location=[35.538377, 129.311359], zoom_start=12)

# # 선택된 시설 위치 마커 추가
# for idx in selected_facilities:
#     folium.Marker(
#         location=[facility_locations[idx, 1], facility_locations[idx, 0]],  # 위도, 경도
#         popup=f"Selected Facility {idx}",
#         icon=folium.Icon(color="red", icon="info-sign")
#     ).add_to(m)

# # 수요 지점 마커 추가
# for idx, coord in enumerate(demand_coordinates):
#     folium.CircleMarker(
#         location=[coord[1], coord[0]],  # 위도, 경도
#         radius=2,
#         popup=f"Demand Point {idx}",
#         color="blue",
#         fill=True,
#         fill_color="blue"
#     ).add_to(m)
    
# # depot 위치 마커 추가 (녹색)
# for idx, depot in enumerate(depot_locations):
#     folium.Marker(
#         location=[depot[1], depot[0]],  # 위도, 경도
#         popup=f"Depot {idx}",
#         icon=folium.Icon(color="green", icon="info-sign")
#     ).add_to(m)

# # 지도 출력
# m

Capacitated MCLP

모델 수정 - 최신 버전

In [22]:
from scipy.spatial.distance import cdist
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import numpy as np
import folium
import geopandas as gpd

# 수요 데이터 (geometry 컬럼 사용)
df_demand = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# 후보 시설 위치 데이터 (클러스터링 결과로 나온 중심 좌표)
facility_locations = np.array(centroids)

# 기존 복지관 위치 데이터 (depot로 처리)
existing_facilities = np.array([
    (129.3299, 35.5299),  # Dosan
    (129.2508, 35.5353),  # Moonsoo
    (129.3187, 35.5140),  # Seonam
    (129.3361, 35.5457)   # Ulsan
])

# 모든 시설 위치: 기존 복지관 + 후보지 위치
all_facilities = np.vstack([existing_facilities, facility_locations])

# 시설 커버 범위
cover_range = 0.01

# 수요 지점 좌표 추출 (위도, 경도)
demand_coordinates = np.array([[point.x, point.y] for point in df_demand.geometry])

# 유클리드 거리 계산 (수요 지점과 시설 위치 간 거리)
distances = cdist(demand_coordinates, all_facilities, metric='euclidean')

# 운영 예산과 용량 설정
total_budget = 2700
facility_capacity = 1500

# 시설 운영 비용 (기존 복지관은 비용을 더 낮게 설정)
facility_cost_existing = 20  # 기존 복지관 비용
facility_cost_new = 200       # 새로운 후보지 비용

# MCLP 모델 설정
model = LpProblem(name="Capacitated_MCLP", sense=LpMaximize)

# 변수 설정
# 기존 복지관은 이미 설치되었으므로 값은 1로 고정하고, 최적화 대상이 아님.
x_existing = [1] * len(existing_facilities)

# 새로운 후보지에 대해서만 변수 설정
x_new = [LpVariable(f"x_{j}", cat="Binary") for j in range(len(facility_locations))]

# 기존 복지관 + 후보지에 대한 x 변수 결합
x = x_existing + x_new

# y_i: 수요 지점 i가 하나 이상의 시설에 의해 커버되면 1, 아니면 0
y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df_demand))]

# z_ij: 수요 지점 i가 시설 j에 할당되면 1, 아니면 0
z = [[LpVariable(f"z_{i}_{j}", cat="Binary") for j in range(len(all_facilities))] for i in range(len(df_demand))]

# 목적 함수: 커버된 수요 지점의 총 수요값을 최대화
model += lpSum(df_demand['demand'][i] * y[i] for i in range(len(df_demand)))

# 제약조건 1: 수요 지점이 커버되려면 그 지점에서 커버 가능한 범위 내의 시설이 하나 이상 설치되어야 함
for i in range(len(df_demand)):
    model += lpSum(z[i][j] for j in range(len(all_facilities)) if distances[i, j] <= cover_range) >= y[i]

# 제약조건 2: 수요 지점 i는 커버 가능 범위 내의 시설 j에만 할당될 수 있음
for i in range(len(df_demand)):
    for j in range(len(all_facilities)):
        if distances[i, j] <= cover_range:
            model += z[i][j] <= x[j]

# 제약조건 3: 각 시설의 용량은 초과되지 않아야 함
for j in range(len(all_facilities)):
    model += lpSum(df_demand['demand'][i] * z[i][j] for i in range(len(df_demand))) <= facility_capacity * x[j]

# 제약조건 4: 총 운영 비용은 예산을 초과하지 않아야 함
# 기존 복지관과 새로운 후보지를 구분하여 설치 비용 계산
model += (lpSum(facility_cost_existing * x[j] for j in range(len(existing_facilities))) +
          lpSum(facility_cost_new * x[j+len(existing_facilities)] for j in range(len(facility_locations)))) <= total_budget

# 제약조건 5: 수요 지점 i는 최대 하나의 시설에만 할당될 수 있음
for i in range(len(df_demand)):
    model += lpSum(z[i][j] for j in range(len(all_facilities))) <= 1

# 모델 최적화
model.solve()

# 결과 출력
# 최적화된 결과에서 선택된 시설만 출력
# 기존 복지관은 이미 선택된 것으로 간주하므로, 새로 최적화된 시설만 확인
selected_facilities = [j for j in range(len(existing_facilities))]  # 기존 복지관은 이미 선택된 상태
selected_facilities += [j + len(existing_facilities) for j in range(len(facility_locations)) if x_new[j].varValue == 1]

covered_demand_points = [i for i in range(len(df_demand)) if y[i].varValue == 1]

# 커버된 수요 비율 계산
total_demand = df_demand['demand'].sum()
covered_demand = sum(df_demand['demand'][i] for i in covered_demand_points)
covered_percentage = (covered_demand / total_demand) * 100

# 각 시설별로 커버한 수요량 계산
facility_coverage = {idx: 0 for idx in selected_facilities}  # 선택된 시설별 커버된 수요량을 저장할 딕셔너리

for i in range(len(df_demand)):
    if y[i].varValue == 1:  # 해당 수요지점이 커버되었을 경우
        for j in range(len(all_facilities)):
            if z[i][j].varValue == 1:  # 해당 수요지점이 시설 j에 할당되었을 경우
                # 해당 시설이 커버한 수요량에 해당 수요지점의 수요량 추가
                if j in selected_facilities:
                    facility_coverage[j] += df_demand['demand'][i]

print("선택된 시설 위치 인덱스 및 좌표:")
for idx in selected_facilities:
    coordinates = all_facilities[idx]
    print(f"인덱스: {idx}, 좌표: (경도: {coordinates[0]}, 위도: {coordinates[1]})")
print("각 시설별 커버된 수요량:")
for idx, coverage in facility_coverage.items():
    coordinates = all_facilities[idx]
    print(f"시설 인덱스: {idx}, 좌표: (경도: {coordinates[0]}, 위도: {coordinates[1]}), 커버된 수요량: {coverage}")
print("전체 수요값: ", total_demand)
print(f"커버된 수요량 비율: {covered_percentage:.2f}%")


# 지도 시각화
m = folium.Map(location=[35.538377, 129.311359], zoom_start=12)

# 기존 복지관 마커 추가 (파란색)
for idx in range(len(existing_facilities)):
    folium.Marker(
        location=[existing_facilities[idx, 1], existing_facilities[idx, 0]],
        popup=f"Existing Facility {idx}",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

# 최적화로 선택된 새로운 시설 위치 마커 추가 (빨간색)
for idx in selected_facilities:
    if idx >= len(existing_facilities):  # 새로운 후보지에 대해서만
        folium.Marker(
            location=[all_facilities[idx, 1], all_facilities[idx, 0]],
            popup=f"Selected Facility {idx}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)

# 수요 지점 마커 추가
for idx, coord in enumerate(demand_coordinates):
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=2,
        popup=f"Demand Point {idx}",
        color="blue",
        fill=True,
        fill_color="blue"
    ).add_to(m)

# 지도 출력
m

선택된 시설 위치 인덱스 및 좌표:
인덱스: 0, 좌표: (경도: 129.3299, 위도: 35.5299)
인덱스: 1, 좌표: (경도: 129.2508, 위도: 35.5353)
인덱스: 2, 좌표: (경도: 129.3187, 위도: 35.514)
인덱스: 3, 좌표: (경도: 129.3361, 위도: 35.5457)
인덱스: 11, 좌표: (경도: 129.32653886435654, 위도: 35.5323393874494)
인덱스: 41, 좌표: (경도: 129.3271448071487, 위도: 35.538883483811134)
인덱스: 45, 좌표: (경도: 129.32260612143162, 위도: 35.53247598621677)
인덱스: 56, 좌표: (경도: 129.27730884933626, 위도: 35.54879619906595)
인덱스: 57, 좌표: (경도: 129.31239017292043, 위도: 35.539695438459965)
인덱스: 71, 좌표: (경도: 129.29696917636892, 위도: 35.534147704479246)
인덱스: 82, 좌표: (경도: 129.3277986797296, 위도: 35.53711667534236)
인덱스: 85, 좌표: (경도: 129.31452557761335, 위도: 35.5273436555103)
인덱스: 86, 좌표: (경도: 129.34377259074262, 위도: 35.5397315341259)
인덱스: 88, 좌표: (경도: 129.32091474461058, 위도: 35.53110010746248)
인덱스: 89, 좌표: (경도: 129.3352287821748, 위도: 35.51769598681901)
인덱스: 100, 좌표: (경도: 129.31555583973363, 위도: 35.536673601668504)
인덱스: 103, 좌표: (경도: 129.2594203073972, 위도: 35.54925746853039)
각 시설별 커버된 수요량:
시설 인덱스: 0, 좌표:

In [23]:
# from scipy.spatial.distance import cdist
# from pulp import LpMaximize, LpProblem, LpVariable, lpSum
# import folium
# import numpy as np
# import geopandas as gpd

# # 수요 데이터 (geometry 컬럼 사용)
# df_demand = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# # 사용자가 지정한 시설 위치 좌표
# facility_coordinates = np.array([
#     [129.3299, 35.5299], [129.2508, 35.5353], [129.3187, 35.514], [129.3361, 35.5457],
#     [129.3335884924386, 35.52279065992096], [129.32185217528098, 35.52945275125455],
#     [129.33269894660782, 35.52929913335067], [129.31346351837473, 35.53752664119595],
#     [129.30541069466506, 35.531438864424835], [129.31693648750422, 35.52682129402336],
#     [129.3284808477662, 35.54425906156538], [129.31981561441708, 35.53522203374411],
#     [129.2672341549313, 35.551842190284084], [129.32637825218657, 35.53118013497816],
#     [129.32334869205377, 35.53869318800164], [129.32588449599737, 35.536952151195024],
#     [129.3316727646812, 35.534419315323845]
# ])

# # 시설 커버 범위
# cover_range = 0.01

# # 수요 지점 좌표 추출 (위도, 경도)
# demand_coordinates = np.array([[point.x, point.y] for point in df_demand.geometry])

# # 유클리드 거리 계산 (수요 지점과 시설 위치 간 거리)
# distances = cdist(demand_coordinates, facility_coordinates, metric='euclidean')

# # 시설 용량 (필요시 변경 가능)
# facility_capacity = 1500

# # MCLP 모델 설정
# model = LpProblem(name="MCLP_Custom_Facilities", sense=LpMaximize)

# # y_i: 수요 지점 i가 하나 이상의 시설에 의해 커버되면 1, 아니면 0
# y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df_demand))]

# # z_ij: 수요 지점 i가 시설 j에 할당되면 1, 아니면 0
# z = [[LpVariable(f"z_{i}_{j}", cat="Binary") for j in range(len(facility_coordinates))] for i in range(len(df_demand))]

# # 목적 함수: 커버된 수요 지점의 총 수요값을 최대화
# model += lpSum(df_demand['demand'][i] * y[i] for i in range(len(df_demand)))

# # 제약조건 1: 수요 지점이 커버되려면 그 지점에서 커버 가능한 범위 내의 시설이 하나 이상 있어야 함
# for i in range(len(df_demand)):
#     model += lpSum(z[i][j] for j in range(len(facility_coordinates)) if distances[i, j] <= cover_range) >= y[i]

# # 제약조건 2: 수요 지점 i는 커버 가능 범위 내의 시설 j에만 할당될 수 있음
# for i in range(len(df_demand)):
#     for j in range(len(facility_coordinates)):
#         if distances[i, j] <= cover_range:
#             model += z[i][j] <= 1

# # 제약조건 3: 각 시설의 용량은 초과되지 않아야 함
# for j in range(len(facility_coordinates)):
#     model += lpSum(df_demand['demand'][i] * z[i][j] for i in range(len(df_demand))) <= facility_capacity

# # 제약조건 4: 수요 지점 i는 최대 하나의 시설에만 할당될 수 있음
# for i in range(len(df_demand)):
#     model += lpSum(z[i][j] for j in range(len(facility_coordinates))) <= 1

# # 모델 최적화
# model.solve()

# # 결과 출력
# covered_demand_points = [i for i in range(len(df_demand)) if y[i].varValue == 1]

# # 커버된 수요 비율 계산
# total_demand = df_demand['demand'].sum()
# covered_demand = sum(df_demand['demand'][i] for i in covered_demand_points)
# covered_percentage = (covered_demand / total_demand) * 100

# # 시설별 커버된 수요량 계산
# facility_coverage = {idx: 0 for idx in range(len(facility_coordinates))}

# for i in range(len(df_demand)):
#     if y[i].varValue == 1:
#         for j in range(len(facility_coordinates)):
#             if z[i][j].varValue == 1:
#                 facility_coverage[j] += df_demand['demand'][i]

# print("각 시설별 커버된 수요량:")
# for idx, coverage in facility_coverage.items():
#     print(f"시설 인덱스: {idx}, 좌표: {facility_coordinates[idx]}, 커버된 수요량: {coverage}")
# print("전체 수요값: ", total_demand)
# print(f"커버된 수요량 비율: {covered_percentage:.2f}%")

# # 지도 시각화
# m = folium.Map(location=[35.538377, 129.311359], zoom_start=12)

# # 시설 위치 마커 추가
# for idx, coord in enumerate(facility_coordinates):
#     folium.Marker(
#         location=[coord[1], coord[0]],
#         popup=f"Facility {idx}",
#         icon=folium.Icon(color="green", icon="info-sign")
#     ).add_to(m)

# # 수요 지점 마커 추가
# for idx, coord in enumerate(demand_coordinates):
#     folium.CircleMarker(
#         location=[coord[1], coord[0]],
#         radius=2,
#         popup=f"Demand Point {idx}",
#         color="blue",
#         fill=True,
#         fill_color="blue"
#     ).add_to(m)

# # 지도 출력
# m

shp 파일 저장

In [24]:
# import geopandas as gpd
# from shapely.geometry import Point

# # 커버 범위를 나타내는 버퍼 반경 (커버 범위 0.01도)
# cover_radius = cover_range  # 기존 커버 범위 사용

# # 시설별 커버 범위를 저장할 리스트
# facility_coverage_data = []

# # 각 시설에 대해 커버 범위 버퍼 생성
# for idx in selected_facilities:
#     facility_point = Point(all_facilities[idx][0], all_facilities[idx][1])  # 시설 좌표 (경도, 위도)
#     facility_buffer = facility_point.buffer(cover_radius)  # 커버 범위를 나타내는 버퍼 생성
    
#     facility_coverage_data.append({
#         "facility_index": idx,         # 시설 인덱스
#         "geometry": facility_buffer    # 버퍼를 geometry로 저장
#     })

# # GeoDataFrame 생성
# gdf_coverage = gpd.GeoDataFrame(facility_coverage_data, crs="EPSG:4326")  # EPSG:4326 좌표계

# # shp 파일로 저장
# output_path_coverage = "facility_coverage_buffers.shp"  # 저장할 파일 경로
# gdf_coverage.to_file(output_path_coverage, driver="ESRI Shapefile")

# print(f"시설 커버 범위를 나타내는 shp 파일이 '{output_path_coverage}'로 저장되었습니다.")

In [25]:
# import geopandas as gpd
# from shapely.geometry import Point

# # 시설별 커버된 수요 지점을 저장할 리스트
# facility_demand_data = []

# # 선택된 시설을 기준으로 커버된 수요 지점 데이터를 정리
# for j in selected_facilities:
#     for i in range(len(df_demand)):
#         if z[i][j].varValue == 1:  # 시설 j가 수요 지점 i를 커버한 경우
#             facility_demand_data.append({
#                 "facility_index": j,                           # 시설 인덱스
#                 "demand_point": (demand_coordinates[i][0], demand_coordinates[i][1]),  # 수요 지점 좌표 (경도, 위도)
#                 "geometry": Point(demand_coordinates[i][0], demand_coordinates[i][1])  # 수요 지점 좌표를 포인트로 변환
#             })

# # GeoDataFrame 생성
# gdf = gpd.GeoDataFrame(facility_demand_data, crs="EPSG:4326")  # EPSG:4326은 WGS84 좌표계

# shp 파일로 저장
# output_path = r"C:\Users\jymin\OneDrive - UNIST (1)\OR LAB\(2024) Orienteering Problem\Code\Data\facility_demand_coverage.shp"  # 저장할 파일 경로
# gdf.to_file(output_path, driver="ESRI Shapefile")

# print(f"shp 파일이 '{output_path}'로 저장되었습니다.")


In [26]:
import geopandas as gpd
from shapely.geometry import Point

# 선택된 시설 위치 좌표 추출
selected_coords = [all_facilities[idx] for idx in selected_facilities]

# F1부터 F14까지 facility_id 지정
facility_ids = [f"F{i}" for i in range(1, len(selected_facilities) + 1)]

# GeoDataFrame 생성
gdf_selected = gpd.GeoDataFrame(
    {'facility_id': facility_ids},  # facility_id를 F1부터 F14까지로 설정
    geometry=[Point(x, y) for x, y in selected_coords],
    crs="EPSG:4326"  # 좌표계 설정 (여기서는 WGS84 사용)
)

gdf_selected

# shapefile 저장
# output_path = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data\result_shp\selected_facilities_v5_F1_format.shp"
# gdf_selected.to_file(output_path, driver="ESRI Shapefile")

,facility_id,geometry
0,F1,POINT (129.3299 35.5299)
1,F2,POINT (129.2508 35.5353)
2,F3,POINT (129.3187 35.514)
3,F4,POINT (129.3361 35.5457)
4,F5,POINT (129.32654 35.53234)
5,F6,POINT (129.32714 35.53888)
6,F7,POINT (129.32261 35.53248)
7,F8,POINT (129.27731 35.5488)
8,F9,POINT (129.31239 35.5397)
9,F10,POINT (129.29697 35.53415)


In [27]:
# import geopandas as gpd
# from shapely.geometry import Point
# import os

# # 시설별 커버된 수요 지점을 저장할 딕셔너리
# facility_demand_dict = {f"F{i}": [] for i in range(1, len(selected_facilities) + 1)}

# # 선택된 시설을 기준으로 커버된 수요 지점 데이터를 정리
# for j_idx, j in enumerate(selected_facilities):  # 선택된 시설 인덱스를 순회
#     facility_id = f"F{j_idx + 1}"  # 시설 ID 설정
#     for i in range(len(df_demand)):
#         if z[i][j].varValue == 1:  # 시설 j가 수요 지점 i를 커버한 경우
#             facility_demand_dict[facility_id].append({
#                 "facility_id": facility_id,  # 시설 ID
#                 "demand_point": (demand_coordinates[i][0], demand_coordinates[i][1]),  # 수요 지점 좌표 (경도, 위도)
#                 "geometry": Point(demand_coordinates[i][0], demand_coordinates[i][1])  # 수요 지점 좌표를 포인트로 변환
#             })

# # 각 시설 ID에 대해 shp 파일로 저장
# output_folder = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data\result_shp"  # 저장 폴더 경로

# for facility_id, demand_data in facility_demand_dict.items():
#     if demand_data:  # 커버된 수요 데이터가 있는 경우에만 저장
#         gdf = gpd.GeoDataFrame(demand_data, crs="EPSG:4326")  # GeoDataFrame 생성
#         print(gdf)  # 디버깅을 위해 출력
#         # output_path = os.path.join(output_folder, f"facility_demand_coverage_{facility_id}.shp")
#         # gdf.to_file(output_path, driver="ESRI Shapefile")  # shp 파일 저장

In [28]:
# import geopandas as gpd
# from shapely.geometry import Point
# import os

# # 시설별 커버된 수요 지점을 저장할 리스트
# facility_demand_data = []

# # 선택된 시설을 기준으로 커버된 수요 지점 데이터를 정리
# for j_idx, j in enumerate(selected_facilities):  # 선택된 시설 인덱스를 순회
#     facility_id = f"F{j_idx + 1}"  # 시설 ID 설정
#     facility_coord = all_facilities[j]  # 시설 좌표 (경도, 위도)
#     for i in range(len(df_demand)):
#         if z[i][j].varValue == 1:  # 시설 j가 수요 지점 i를 커버한 경우
#             facility_demand_data.append({
#                 "facility_id": facility_id,  # 시설 ID
#                 "facility_geometry": Point(facility_coord[0], facility_coord[1]),  # 시설 좌표의 Point
#                 "demand_geometry": Point(demand_coordinates[i][0], demand_coordinates[i][1])  # 수요 지점 좌표의 Point
#             })

# # GeoDataFrame 생성
# gdf = gpd.GeoDataFrame(
#     facility_demand_data,
#     geometry="demand_geometry",  # 최종 geometry 필드는 수요 지점 좌표로 설정
#     crs="EPSG:4326"  # WGS84 좌표계 사용
# )

# # 열 이름을 명확하게 설정
# gdf.rename(columns={"geometry": "demand_geometry"}, inplace=True)
# gdf.head()

# # 저장
# # output_folder = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data\result_shp"
# # output_path = os.path.join(output_folder, "all_facility_demand_coverage.shp")
# # gdf.to_file(output_path, driver="ESRI Shapefile")

In [29]:
# 시설 F9만 포함된 GeoDataFrame 생성
gdf_facility_i = gdf_selected[gdf_selected['facility_id'] == 'F3']

# Shapefile 저장 (시설 F9만)
output_path_i = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data\result_shp\selected_facilities_F3.shp"
gdf_facility_i.to_file(output_path_i, driver="ESRI Shapefile")

C:\Users\jymin\AppData\Local\Temp\ipykernel_9632\2850347476.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_facility_i.to_file(output_path_i, driver="ESRI Shapefile")
c:\Users\jymin\AppData\Local\Programs\Python\Python39\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'facility_id' to 'facility_i'
  ogr_write(


Budget vs Population Coverage

In [30]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.spatial.distance import cdist
# from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# # 기존 코드에서 사용한 데이터 및 변수들 정의
# # 기존 복지관 위치, 새로운 후보지 위치, 수요 지점, 커버 범위 등
# existing_facilities = np.array([
#     (129.3299, 35.5299),  # Dosan
#     (129.2508, 35.5353),  # Moonsoo
#     (129.3187, 35.5140),  # Seonam
#     (129.3361, 35.5457)   # Ulsan
# ])
# facility_locations = np.array(centroids)
# all_facilities = np.vstack([existing_facilities, facility_locations])
# demand_coordinates = np.array([[point.x, point.y] for point in df_demand.geometry])
# distances = cdist(demand_coordinates, all_facilities, metric='euclidean')

# # 시뮬레이션할 예산 목록 설정
# budget_list = np.linspace(1000, 3000, 100)
# covered_percentages = []

# # 커버 범위 및 수요 설정
# cover_range = 0.01
# facility_capacity = 1500
# facility_cost_existing = 20
# facility_cost_new = 200

# # 각 예산에 대해 최적화 수행
# for budget in budget_list:
#     # MCLP 모델 설정
#     model = LpProblem(name="Capacitated_MCLP", sense=LpMaximize)

#     # 기존 복지관 변수 설정 (이미 설치된 것으로 가정)
#     x_existing = [1] * len(existing_facilities)
#     x_new = [LpVariable(f"x_{j}", cat="Binary") for j in range(len(facility_locations))]
#     x = x_existing + x_new
#     y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df_demand))]
#     z = [[LpVariable(f"z_{i}_{j}", cat="Binary") for j in range(len(all_facilities))] for i in range(len(df_demand))]

#     # 목적 함수: 커버된 수요값 최대화
#     model += lpSum(df_demand['demand'][i] * y[i] for i in range(len(df_demand)))

#     # 제약조건 추가 (수요 커버, 할당, 용량, 비용)
#     for i in range(len(df_demand)):
#         model += lpSum(z[i][j] for j in range(len(all_facilities)) if distances[i, j] <= cover_range) >= y[i]
#     for i in range(len(df_demand)):
#         for j in range(len(all_facilities)):
#             if distances[i, j] <= cover_range:
#                 model += z[i][j] <= x[j]
#     for j in range(len(all_facilities)):
#         model += lpSum(df_demand['demand'][i] * z[i][j] for i in range(len(df_demand))) <= facility_capacity * x[j]
#     model += (lpSum(facility_cost_existing * x[j] for j in range(len(existing_facilities))) +
#               lpSum(facility_cost_new * x[j+len(existing_facilities)] for j in range(len(facility_locations)))) <= budget
#     for i in range(len(df_demand)):
#         model += lpSum(z[i][j] for j in range(len(all_facilities))) <= 1

#     # 최적화 실행
#     model.solve()

#     # 커버된 수요 계산
#     covered_demand_points = [i for i in range(len(df_demand)) if y[i].varValue == 1]
#     total_demand = df_demand['demand'].sum()
#     covered_demand = sum(df_demand['demand'][i] for i in covered_demand_points)
#     covered_percentage = (covered_demand / total_demand) * 100
#     covered_percentages.append(covered_percentage)

# # 그래프 그리기
# plt.plot(budget_list, covered_percentages, 'o-')
# plt.xlabel('Budget ($)')
# plt.ylabel('Population Coverage (%)')
# plt.title('Budget vs Population Coverage')
# plt.grid(True)
# plt.show()


잘 되는 기존꺼

In [31]:
# from scipy.spatial.distance import cdist
# from pulp import LpMaximize, LpProblem, LpVariable, lpSum
# import folium
# import numpy as np
# import geopandas as gpd

# # 수요 데이터 (geometry 컬럼 사용)
# df_demand = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# # 후보 시설 위치 데이터 (클러스터링 결과로 나온 중심 좌표)
# facility_locations = np.array(centroids)

# # 시설 커버 범위 (~~정도의 거리 내에 있는 지점들을 커버한다고 가정)
# cover_range = 0.005 #0.006까지 가능

# # GeoDataFrame에서 좌표 추출 (수요 지점의 위도, 경도)
# demand_coordinates = np.array([[point.x, point.y] for point in df_demand.geometry])

# # 유클리드 거리 계산 (수요 지점과 후보 시설 위치 간 거리)
# distances = cdist(demand_coordinates, facility_locations, metric='euclidean')

# # 예산과 용량 설정
# total_budget = 2000  # 총 예산
# facility_capacity = 1500  # 각 시설의 용량
# facility_cost = 200  # 각 시설의 설치 비용

# # 기존 복지관 depot 위치 데이터
# depot_locations = [
#     (129.3299, 35.5299),  # df_Dosan의 Depot 좌표
#     (129.2508, 35.5353),  # df_Moonsoo의 Depot 좌표
#     (129.3187, 35.5140),  # df_Seonam의 Depot 좌표
#     (129.3361, 35.5457)   # df_Ulsan의 Depot 좌표
# ]

# # 기존 복지관과 수요 지점 사이의 거리 계산
# depot_locations = np.array(depot_locations)
# depot_distances = cdist(demand_coordinates, depot_locations, metric='euclidean')

# # 기존 복지관에 의해 커버되는 수요 지점을 저장할 리스트
# covered_by_existing_depots = np.zeros(len(df_demand), dtype=bool)

# # 기존 복지관 커버 범위 내에 있는 수요 지점들을 True로 표시
# for i in range(len(df_demand)):
#     if any(depot_distances[i, j] <= cover_range for j in range(len(depot_locations))):
#         covered_by_existing_depots[i] = True

# # MCLP 모델 설정
# model = LpProblem(name="Capacitated_MCLP", sense=LpMaximize)

# # 변수 설정
# # x_j: 후보 시설 위치 j에 시설이 설치되면 1, 아니면 0
# x = [LpVariable(f"x_{j}", cat="Binary") for j in range(len(facility_locations))]

# # y_i: 수요 지점 i가 하나 이상의 시설에 의해 커버되면 1, 아니면 0
# y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df_demand))]

# # z_ij: 수요 지점 i가 시설 j에 할당되면 1, 아니면 0
# z = [[LpVariable(f"z_{i}_{j}", cat="Binary") for j in range(len(facility_locations))] for i in range(len(df_demand))]

# # 목적 함수: 커버된 수요 지점의 총 수요값을 최대화
# model += lpSum(df_demand['demand'][i] * y[i] for i in range(len(df_demand)))

# # 제약조건 1: 수요 지점이 커버되려면 그 지점에서 커버 가능한 범위 내의 시설이 하나 이상 설치되어야 함
# for i in range(len(df_demand)):
#     model += lpSum(z[i][j] for j in range(len(facility_locations)) if distances[i, j] <= cover_range) >= y[i]
    

# # 제약조건 2: 수요 지점 i는 커버 가능 범위 내의 시설 j에만 할당될 수 있음
# for i in range(len(df_demand)):
#     for j in range(len(facility_locations)):
#         if distances[i, j] <= cover_range:
#             model += z[i][j] <= x[j]

# # 제약조건 3: 각 시설의 용량은 초과되지 않아야 함
# for j in range(len(facility_locations)):
#     model += lpSum(df_demand['demand'][i] * z[i][j] for i in range(len(df_demand))) <= facility_capacity * x[j]

# # 제약조건 4: 총 설치 비용은 예산을 초과하지 않아야 함
# model += lpSum(facility_cost * x[j] for j in range(len(facility_locations))) <= total_budget


# # 추가 제약조건: 기존 복지관에 의해 이미 커버된 수요 지점은 새로운 시설이 중복 커버하지 않도록 설정
# for i in range(len(df_demand)):
#     if covered_by_existing_depots[i]:  # 기존 복지관에 의해 커버된 수요 지점
#         model += y[i] == 0  # 이 수요 지점은 새로운 시설에 의해 커버되지 않도록 설정
        
# # 모델 최적화
# model.solve()

# # 결과 출력
# selected_facilities = [j for j in range(len(facility_locations)) if x[j].varValue == 1]
# covered_demand_points = [i for i in range(len(df_demand)) if y[i].varValue == 1]


# # 전체 수요값을 기준으로 커버된 수요값을 합산
# total_demand = df_demand['demand'].sum()  # 전체 수요값

# # 기존 복지관이 커버한 수요값과 새로운 시설이 커버한 수요값을 합산
# covered_demand = sum(df_demand['demand'][i] for i in range(len(df_demand)) if covered_by_existing_depots[i] or i in covered_demand_points)

# # 커버된 수요값 비율 계산 (기존 복지관과 새로운 시설의 커버 범위 합산)
# covered_percentage = (covered_demand / total_demand) * 100

# # # 커버된 수요 지점 비율 계산
# # total_demand_points = len(df_demand)
# # covered_percentage = (len(covered_demand_points) / total_demand_points) * 100

# print("선택된 시설 위치 인덱스: ", selected_facilities)
# print("선택된 시설 위치 좌표: ", facility_locations[selected_facilities])
# print("전체 수요값: ", total_demand)
# print(f"커버된 수요량 비율: {covered_percentage:.2f}%")


# # 지도 시각화
# # 지도 초기 중심
# m = folium.Map(location=[35.538377, 129.311359], zoom_start=12)

# # 선택된 시설 위치 마커 추가
# for idx in selected_facilities:
#     folium.Marker(
#         location=[facility_locations[idx, 1], facility_locations[idx, 0]],  # 위도, 경도
#         popup=f"Selected Facility {idx}",
#         icon=folium.Icon(color="red", icon="info-sign")
#     ).add_to(m)

# # 수요 지점 마커 추가
# for idx, coord in enumerate(demand_coordinates):
#     folium.CircleMarker(
#         location=[coord[1], coord[0]],  # 위도, 경도
#         radius=2,
#         popup=f"Demand Point {idx}",
#         color="blue",
#         fill=True,
#         fill_color="blue"
#     ).add_to(m)
    
# # depot 위치 마커 추가 (녹색)
# for idx, depot in enumerate(depot_locations):
#     folium.Marker(
#         location=[depot[1], depot[0]],  # 위도, 경도
#         popup=f"Depot {idx}",
#         icon=folium.Icon(color="green", icon="info-sign")
#     ).add_to(m)

# # 지도 출력
# m


기존 복지관 운영 비용 추가 코드

In [32]:
# from scipy.spatial.distance import cdist
# from pulp import LpMaximize, LpProblem, LpVariable, lpSum
# import folium
# import numpy as np
# import geopandas as gpd

# # 수요 데이터 (geometry 컬럼 사용)
# df_demand = gpd.GeoDataFrame(df_Namgu, geometry='geometry')

# # 후보 시설 위치 데이터 (클러스터링 결과로 나온 중심 좌표)
# facility_locations = np.array(centroids)

# # 시설 커버 범위 (~~정도의 거리 내에 있는 지점들을 커버한다고 가정)
# cover_range = 0.01

# # GeoDataFrame에서 좌표 추출 (수요 지점의 위도, 경도)
# demand_coordinates = np.array([[point.x, point.y] for point in df_demand.geometry])

# # 유클리드 거리 계산 (수요 지점과 후보 시설 위치 간 거리)
# distances = cdist(demand_coordinates, facility_locations, metric='euclidean')

# # 운영 예산과 용량 설정
# total_budget = 1100  # 총 예산
# facility_capacity = 1500  # 각 시설의 용량
# facility_cost_new = 200  # 새로운 시설의 운영 비용
# facility_cost_existing = 20  # 기존 복지관의 운영 비용

# # 기존 복지관 depot 위치 데이터
# depot_locations = [
#     (129.3299, 35.5299),  # df_Dosan의 Depot 좌표
#     (129.2508, 35.5353),  # df_Moonsoo의 Depot 좌표
#     (129.3187, 35.5140),  # df_Seonam의 Depot 좌표
#     (129.3361, 35.5457)   # df_Ulsan의 Depot 좌표
# ]

# # 기존 복지관과 수요 지점 사이의 거리 계산
# depot_locations = np.array(depot_locations)
# depot_distances = cdist(demand_coordinates, depot_locations, metric='euclidean')

# # 기존 복지관에 의해 커버되는 수요 지점을 저장할 리스트
# covered_by_existing_depots = np.zeros(len(df_demand), dtype=bool)

# # 기존 복지관 커버 범위 내에 있는 수요 지점들을 True로 표시
# for i in range(len(df_demand)):
#     if any(depot_distances[i, j] <= cover_range for j in range(len(depot_locations))):
#         covered_by_existing_depots[i] = True

# # MCLP 모델 설정
# model = LpProblem(name="Capacitated_MCLP", sense=LpMaximize)

# # 변수 설정
# # x_j: 후보 시설 위치 j에 시설이 설치되면 1, 아니면 0
# x = [LpVariable(f"x_{j}", cat="Binary") for j in range(len(facility_locations))]

# # y_i: 수요 지점 i가 하나 이상의 시설에 의해 커버되면 1, 아니면 0
# y = [LpVariable(f"y_{i}", cat="Binary") for i in range(len(df_demand))]

# # z_ij: 수요 지점 i가 시설 j에 할당되면 1, 아니면 0
# z = [[LpVariable(f"z_{i}_{j}", cat="Binary") for j in range(len(facility_locations))] for i in range(len(df_demand))]

# # 목적 함수: 커버된 수요 지점의 총 수요값을 최대화
# model += lpSum(df_demand['demand'][i] * y[i] for i in range(len(df_demand)))

# # 제약조건 1: 수요 지점이 커버되려면 그 지점에서 커버 가능한 범위 내의 시설이 하나 이상 설치되어야 함
# for i in range(len(df_demand)):
#     model += lpSum(z[i][j] for j in range(len(facility_locations)) if distances[i, j] <= cover_range) >= y[i]

# # 제약조건 2: 수요 지점 i는 커버 가능 범위 내의 시설 j에만 할당될 수 있음
# for i in range(len(df_demand)):
#     for j in range(len(facility_locations)):
#         if distances[i, j] <= cover_range:
#             model += z[i][j] <= x[j]

# # 제약조건 3: 각 시설의 용량은 초과되지 않아야 함
# for j in range(len(facility_locations)):
#     model += lpSum(df_demand['demand'][i] * z[i][j] for i in range(len(df_demand))) <= facility_capacity * x[j]

# # 제약조건 4: 총 설치 비용은 예산을 초과하지 않아야 함
# # 기존 복지관의 운영 비용과 새로운 시설의 설치 비용을 합산하여 예산 제약 추가
# model += (lpSum(facility_cost_existing for _ in depot_locations) + 
#           lpSum(facility_cost_new * x[j] for j in range(len(facility_locations)))) <= total_budget

# # 추가 제약조건: 기존 복지관에 의해 이미 커버된 수요 지점은 새로운 시설이 중복 커버하지 않도록 설정
# for i in range(len(df_demand)):
#     if covered_by_existing_depots[i]:  # 기존 복지관에 의해 커버된 수요 지점
#         model += y[i] == 0  # 이 수요 지점은 새로운 시설에 의해 커버되지 않도록 설정

# # 모델 최적화
# model.solve()

# # 결과 출력
# selected_facilities = [j for j in range(len(facility_locations)) if x[j].varValue == 1]
# covered_demand_points = [i for i in range(len(df_demand)) if y[i].varValue == 1]

# # 전체 수요값을 기준으로 커버된 수요값을 합산
# total_demand = df_demand['demand'].sum()  # 전체 수요값

# # 기존 복지관이 커버한 수요값과 새로운 시설이 커버한 수요값을 합산
# covered_demand = sum(df_demand['demand'][i] for i in range(len(df_demand)) if covered_by_existing_depots[i] or i in covered_demand_points)

# # 커버된 수요값 비율 계산 (기존 복지관과 새로운 시설의 커버 범위 합산)
# covered_percentage = (covered_demand / total_demand) * 100

# print("선택된 시설 위치 인덱스: ", selected_facilities)
# print("선택된 시설 위치 좌표: ", facility_locations[selected_facilities])
# print("전체 수요값: ", total_demand)
# print(f"커버된 수요량 비율: {covered_percentage:.2f}%")

# # 지도 시각화
# m = folium.Map(location=[35.538377, 129.311359], zoom_start=12)

# # 선택된 시설 위치 마커 추가
# for idx in selected_facilities:
#     folium.Marker(
#         location=[facility_locations[idx, 1], facility_locations[idx, 0]],
#         popup=f"Selected Facility {idx}",
#         icon=folium.Icon(color="red", icon="info-sign")
#     ).add_to(m)

# # 수요 지점 마커 추가
# for idx, coord in enumerate(demand_coordinates):
#     folium.CircleMarker(
#         location=[coord[1], coord[0]],
#         radius=2,
#         popup=f"Demand Point {idx}",
#         color="blue",
#         fill=True,
#         fill_color="blue"
#     ).add_to(m)

# # depot 위치 마커 추가 (녹색)
# for idx, depot in enumerate(depot_locations):
#     folium.Marker(
#         location=[depot[1], depot[0]],  # 위도, 경도
#         popup=f"Depot {idx}",
#         icon=folium.Icon(color="green", icon="info-sign")
#     ).add_to(m)

# # 지도 출력
# m